**Check GPU connection**


In [ ]:
!nvidia-smi

Tue Mar 16 17:29:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Import all required libraries**


In [1]:
import datetime
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop, Rescaling, RandomFlip, RandomRotation, Resizing
from tensorflow.keras.backend import expand_dims
from tensorflow.keras import layers
from cv2 import cv2 as cv

**Create folder for accuracy/loss charts and model summary**

In [27]:
print("Input folder name:")
name = input()
!mkdir $name

Input folder name:
r6


**Convolutional Neural Network (CNN) with 4 conv layers**

In [28]:
def train(folder_name):

    model_path = './{}'.format(folder_name)
    shape = (300, 300, 3)
    batch_size = 64

    (train, test), info = tfds.load(
        "rock_paper_scissors",
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )

    def normalize_img(image, label):
        """Normalizes images: `uint8` -> `float32`."""
        return tf.cast(image, tf.float32) / 255., label

    # Pipeline for training dataset
    train = train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    train = train.cache()
    train = train.shuffle(info.splits['train'].num_examples)
    train = train.batch(batch_size)
    train = train.prefetch(tf.data.experimental.AUTOTUNE)

    # Pipeline for testing dataset
    test = test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    test = test.batch(batch_size)
    test = test.cache()
    test = test.prefetch(tf.data.experimental.AUTOTUNE)
    

    data_augmentation = keras.Sequential([
        RandomFlip("horizontal_and_vertical"),
        RandomRotation(0.1)
    ])

    resize_and_rescale = keras.Sequential([
    Resizing(150, 150),
    Rescaling(1./255)
    ])

    # Input layer
    inputs = keras.Input(shape=shape)

    x = layers.Conv2D(32, (3, 3), padding='same', activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(32, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(64, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(128, (3, 3), activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(256, activation="relu")(x)
    outputs = layers.Dense(3, activation="softmax")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    # Displays model summary in terminal
    model.summary()

    # Sets loss and optomazer algoriths
    model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Tensorboard
    log_dir = '/{}/'.format(name) + "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Trains the model and saves it do history variable
    history = model.fit(train, batch_size=batch_size,
                          epochs=10, validation_data=test, callbacks=[tensorboard_callback])

    # Evaluate
    model.evaluate(test, batch_size=batch_size)

    # Saves model
    keras.models.save_model(
        model,
        model_path,
        overwrite=True,
        include_optimizer=True,
        save_format=None,
        signatures=None,
        options=None,
    )

train(name)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 300, 300, 3)]     0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 300, 300, 32)      896       
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 150, 150, 32)      0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 148, 148, 32)      9248      
_________________________________________________________________
max_pooling2d_49 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_50 (MaxPooling (None, 36, 36, 64)        0  

INFO:tensorflow:Assets written to: ./r6/assets


In [29]:
!zip -r /content/r6.zip /content/r6/
from google.colab import files
files.download("/content/r6.zip")


  adding: content/r6/ (stored 0%)
  adding: content/r6/saved_model.pb (deflated 90%)
  adding: content/r6/assets/ (stored 0%)
  adding: content/r6/variables/ (stored 0%)
  adding: content/r6/variables/variables.index (deflated 70%)
  adding: content/r6/variables/variables.data-00000-of-00001 (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>